In [1]:
import json
import requests
import popelines
import os
from datetime import datetime
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
    

In [2]:
from generate_original_with_timeseries import read_endpoint
# , generate_projects, generate_experiments

In [3]:
def generate_results(results_jayson):  
    flattened_j_ts = []

    for metric in results_jayson['metrics']:

        if 'results' in metric.keys():
            flattened_results = []
            for ts in metric['results']:
                flattened_timeseries = []
                ts['upload_ts'] = str(datetime.utcnow())
                flattened_timeseries.append(flatten(ts, {}, ''))

                # Replace old 'metrics' with new 'flattened_results'
                update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_timeseries, destination_key='results')
                flattened_results.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

        else:
            flattened_results = [flatten(results_jayson, {}, '')]

        update_new_j_ts = populating_vals(outer_dict=results_jayson, inner_flattened_list=flattened_results, destination_key='metrics')
        flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))

    return flattened_j_ts



In [4]:
# ############################################### Keys and Authentication #######################################
# if not os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS'):
#     os.environ['GOOGLE_ACCOUNT_CREDENTIALS'] = '/home/engineering/keyfile.json'
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

# directory = str(os.path.abspath(os.path.dirname(__file__)))
directory = os.getcwd()

############################################### Instantiating Popelines #######################################
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)


In [5]:
# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

start_time = '2019-01-20T00:00:00.000Z'

end_time = '2019-02-20T00:00:00.000Z'


In [6]:
params = (
#     ('project_id', 13655420480),
    ('per_page', 100),
    ('start_time', start_time),
    ('end_time', end_time),
) 

In [7]:
# experiment_id_list = [13559460138, 12746930280]
experiment_id_list = [12746930280]

In [8]:
for experiment_id in experiment_id_list:
    result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/results'
    response_ts = read_endpoint(result_endpoint, headers, params)
    print(f"got experiment {experiment_id}")
    # if '' then the experiment has not started yet
    if response_ts == '' or 'bad' in response_ts:
        j_ts = {'experiment_id': experiment_id}
        new_j_ts = j_ts
        new_j_ts['upload_ts'] = str(datetime.utcnow())
        new_j_ts['metric_calculating_ts'] = end_time
#         pope.write_to_json(file_name=f'{directory}/../uploads/no_results.json', jayson=[new_j_ts], mode='w')
#         pope.write_to_bq(table_name='results', file_name=f'{directory}/../uploads/no_results.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)

    else:

        new_j_ts = pope.fix_json_values(callback=fix_values, obj=response_ts, reset_key='results')
        flattened_j_ts = generate_results(new_j_ts)

#         pope.write_to_json(file_name=f'{directory}/../uploads/results.json', jayson=flattened_j_ts, mode='w')
#         pope.write_to_bq(table_name='results', file_name=f'{directory}/../uploads/results.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
#         print(f"Successfully uploaded result time series for experiment {experiment_id}")

got experiment 12746930280


In [9]:
flattened_j_ts

[{'confidence_threshold': 0.95,
  'end_time': '2019-02-20T00:00:00.000000Z',
  'experiment_id': 12746930280,
  'metrics_aggregator': 'unique',
  'metrics_event_id': 12826470463,
  'metrics_name': 'CTA - View Guide Now',
  'metrics_results_is_baseline': False,
  'metrics_results_level': 'variation',
  'metrics_results_lift_is_significant': False,
  'metrics_results_lift_lift_status': 'better',
  'metrics_results_lift_significance': 0.5478614851902412,
  'metrics_results_lift_value': 0.120905054437078,
  'metrics_results_lift_visitors_remaining': 1609,
  'metrics_results_name': 'Variation #1',
  'metrics_results_rate': 0.2701219512195122,
  'metrics_results_results_id': '12762280184',
  'metrics_results_samples': 1640,
  'metrics_results_upload_ts': '2019-02-28 21:06:11.362144',
  'metrics_results_value': 443.0,
  'metrics_results_variance': 0.1971560826888757,
  'metrics_results_variation_id': '12762280184',
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'r